## 准备数据

In [26]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [27]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [ ]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.truncated_normal([28*28, 128], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([128]))
        self.W2 = tf.Variable(tf.random.truncated_normal([128, 10], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 28*28])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits

model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [29]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [30]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.5492089 ; accuracy 0.0828
epoch 1 : loss 2.5276842 ; accuracy 0.08601667
epoch 2 : loss 2.5072677 ; accuracy 0.08936667
epoch 3 : loss 2.487867 ; accuracy 0.09335
epoch 4 : loss 2.4693954 ; accuracy 0.098
epoch 5 : loss 2.4517736 ; accuracy 0.10305
epoch 6 : loss 2.4349353 ; accuracy 0.108616665
epoch 7 : loss 2.4188151 ; accuracy 0.11365
epoch 8 : loss 2.4033551 ; accuracy 0.120066665
epoch 9 : loss 2.3885074 ; accuracy 0.12598333
epoch 10 : loss 2.3742218 ; accuracy 0.13216667
epoch 11 : loss 2.3604562 ; accuracy 0.13816667
epoch 12 : loss 2.3471708 ; accuracy 0.14483333
epoch 13 : loss 2.3343306 ; accuracy 0.15095
epoch 14 : loss 2.3219025 ; accuracy 0.15721667
epoch 15 : loss 2.309858 ; accuracy 0.16345
epoch 16 : loss 2.2981708 ; accuracy 0.17025
epoch 17 : loss 2.2868156 ; accuracy 0.17633334
epoch 18 : loss 2.2757704 ; accuracy 0.18225
epoch 19 : loss 2.2650135 ; accuracy 0.18798333
epoch 20 : loss 2.2545276 ; accuracy 0.19386667
epoch 21 : loss 2.244294 ; accur